<a href="https://colab.research.google.com/github/Adityabhaskar685/langchain/blob/main/fineTuning_flacon7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers --q
!pip install xformers --q
!pip install einops --q
!pip install accelerate --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 93.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 12.5 MB/s eta 0:00:00


In [4]:
!pip install datasets --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.9 MB/s eta 0:00:00


In [9]:
!pip install peft --q
!pip install trl --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 4.3 MB/s eta 0:00:00


In [36]:
# required libraries

from dataclasses import dataclass, field
from typing import Optional

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
)

from peft.tuners.lora import LoraLayer
from trl import SFTTrainer

In [13]:
#Define and Parse Argument



In [42]:
@dataclass
class ScriptArguments:
  """
  These arguments vary depending on how many GPU you have, what their capacity and features are,
  and what size model you want to train.
  """

  local_rank : Optional[int] = field(default = 1, metadata = {'help': "Used for multi-gpu"})

  per_device_train_batch_size: Optional[int] = field(default = 4)
  per_device_eval_batch_size: Optional[int] = field(default = 1)
  gradient_accumulation_steps: Optional[int] = field(default = 4)
  learning_rate: Optional[float] = field(default = 2e-4)
  max_grad_norm: Optional[float] = field(default = 0.3)
  weight_decay: Optional[int] = field(default = 0.001)
  lora_alpha: Optional[int] = field(default = 16)
  lora_dropout: Optional[float] = field(default = 0.1)
  lora_r: Optional[int] = field(default = 64)
  max_seq_length: Optional[int] = field(default = 512)
  model_name : Optional[str] = field(
      default = 'tiiuae/falcon-7b',
      metadata= {
          'help': "The model that you want to train from the hugging Face hub. E.g gpt2, gpt2-x1, bert ,etc."

      },
  )

  dataset_name: Optional[str] = field(
      default = "timdettmers/openassistant-guanaco",
      metadata = {'help' : "Thre preference dataset to use."},
  )

  use_4bit: Optional[bool] = field(
      default = True,
      metadata = {"help": "Activate 4bit precision base model loading"},

  )
  use_nested_quant: Optional[bool] = field(
      default = False,
      metadata = {"help": "Activate nested quantization for 4bit base models"}

  )
  bnb_4bit_compute_dtype : Optional[str] = field(
      default = 'float16',
      metadata = {"help": 'Compute dtype for 4bit base models'},
  )
  bnb_4bit_quant_type : Optional[str] = field(
      default = 'nf4',
      metadata = {"help": 'Quantization type fp4 or nf4'},
  )
  num_train_epochs : Optional[int] = field(
      default = 1,
      metadata = {"help": "The number of training epochs for the reward model."}
  )
  fp16: Optional[bool] = field(
      default = False,
      metadata = {'help' : 'Enables fp16 training.'},
  )

  bf16: Optional[bool] = field(
      default = False,
      metadata = {"help" : "Enables bf16 training."}
  )

  packing : Optional[bool] = field(
      default = False,
      metadata = {"help": 'Use packing dataset creating.'}
  )

  gradient_checkpointing: Optional[bool] = field(
      default = True,
      metadata = {"help": "Enables gradient checkpointing"}
  )
  optim : Optional[str] = field(
      default = "paged_adamw_32bit",
      metadata = {'help' : "The optimizer to use."},

  )

  lr_schedular_type : str = field(
      default = "constant",
      metadata = {'help': 'Learning rate schedule. Constant a bit better than cosine, and has advantage for analysis'},

  )

  max_step: int = field(
      default = 10000,
      metadata = {'help': 'How many optimizer update steps to take.'}
  )

  warmup_ratio: int = field(default = 0.03, metadata = {'help' : 'Fraction of steps to do a warmup for'})
  group_by_length: bool = field(
      default = True,
      metadata = {
          'help' : 'Group sequence into batches with same length. Saves memory and speeds up training considerabley.'
      }
  )
  save_steps: int = field(default = 10, metadata = {"help": "Save checkpoint every X updates steps."})
  logging_steps : int = field(default = 10, metadata = {"help": 'Log every X updates steps'})



parser = HfArgumentParser(ScriptArguments)
script_args = parser.parse_args_into_dataclasses()[0]



╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 104>:104                                                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/transformers/hf_argparser.py:355 in                      │
│ parse_args_into_dataclasses                                                                      │
│                                                                                                  │
│   352 │   │   │   return (*outputs, remaining_args)                                              │
│   353 │   │   else:                                                                              │
│   354 │   │   │   if remaining_args:                                                             │
│ ❱ 355 │   │   │   │   raise ValueError(f"Some specified arguments are not used by the HfArgume   │
│   356 │   │   │                                                                                  │
│   357 │   │   │   return (*outputs,)                                                             │
│   358                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Some specified arguments are not used by the HfArgumentParser: ['-f', 
'/root/.local/share/jupyter/runtime/kernel-e3c4d980-e845-4baa-9d8b-4598633ab531.json']

In [29]:
def create_and_prepare_model(args):
  compute_dtype = getattr(torch.args.bnd_4bit_compute_dtype)

  bnd_config = BitsAndBytesConfig(
      load_in_4bit = args.use_4bit,
      bnb_4bit_quant_type = args.bnb_4bit_quant_type,
      bnb_4bit_compute_dtype= compute_dtype,
      bnb_4bit_use_double_quant = args.use_nested_quant
  )

  if compute_dtype == torch.float16 and args.use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
      print("=" * 80)
      print("Your GPU support bfloat16, you can accelerate training with the argument --bf16")
      print("=" * 80)
    device_map = {"": 0}

    model = AutoModelForCausalLM.from_pretrained(
        args.model_name,
        quantization_config = bnd_config,
        device_map = device_map,
        trust_remote_code = True
    )

    peft_config = LoraConfig(
        lora_alpha = script_args.lora.alpha,
        lora_dropout = script_args.lora_dropout,
        r = script_args.lora_r,
        bias = 'none',
        task_type = 'CAUSAL_LM',
        target_modules = [
            'query_key_value',
            'dense',
            'dense_h_to_4h',
            'dense_4h_to_h'
        ], # "word_embedding", 'lm_head'
    )

    tokenizer = AutoTokenizer.from_pretrained(script_args.model_name,
                                              trust_remote_code = True,)
    tokenizer.pad_token = tokenizer.eos_token

    return model, peft_config, tokenizer

In [33]:
training_argument = TrainingArguments(
    outdir = './results',
    per_device_train_batch_size = script_args.per_device_train_batch_size,
    gradient_accumulation_steps = script_args.gradient_accumulation_steps,
    optim = script_args.optim,
    save_steps = script_args.save_steps,
    logging_steps = script_args.logging_steps,
    learning_rate = script_args.learning_rate,
    fp16 = script_args.fp16,
    bf16 = script_args.bf16,
    max_grad_norm = script_args.max_grad_norm,
    max_steps = script_args.max_steps,
    warmup_ratio = script_args.warmup_ratio,
    group_by_length = script_args.group_by_length,
    lr_schedular_type = script_args.lr_scheduler_type,
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:3                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'script_args' is not defined

In [32]:
model,peft_config, tokenizer = create_and_prepare_model(script_args)
model.config.use_cache = False
dataset = load_dataset(script_args,dataset_name, split = 'train')

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    peft_config = peft_config,
    dataset_text_field = 'text',
    max_seq_length = script_args.max_seq_length,
    tokenizer = tokenizer,
    args = training_arguments,
    packing = script_args.packing
)

for name, module in trainer.model.named_modules():
  if isinstance(module, LoraLayer):
    if script_args.bf16:
      module = module.to(torch.bfloat16)
  if 'norm' in name:
    module = module.to(torch.float32)
  if "lm_head" in name or 'embed_tokens' in name:
    if hasattr(module, 'weight'):
      if script_args.bf16 and module.weight.dtype == torch.float32:
        module = module.to(torch.bfloat32)
trainer.train()

/content
